In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_inference_server.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/__init__.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/templates.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/base_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/relay.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/__init__.py
/kaggle/input/cmi-detect-behav

# Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import classification_report,accuracy_score
from scipy.stats import skew,kurtosis,iqr

#Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.utils import to_categorical

2025-07-05 16:10:46.372141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751731846.540011      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751731846.587881      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
target_behaviors = ['bfrb_type_A', 'bfrb_type_B']


IMU_SENSOR_COLUMNS = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x', 'mag_y', 'mag_z']
THERMOPILE_SENSOR_COLUMNS = ['therm_1', 'therm_2', 'therm_3', 'therm_4']
TOF_SENSOR_COLUMNS = ['tof_distance', 'tof_signal_strength'] 

ALL_EXPECTED_SENSOR_COLUMNS = IMU_SENSOR_COLUMNS + THERMOPILE_SENSOR_COLUMNS + TOF_SENSOR_COLUMNS

In [4]:
df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv')

In [5]:
df.columns[1:50]

Index(['sequence_type', 'sequence_id', 'sequence_counter', 'subject',
       'orientation', 'behavior', 'phase', 'gesture', 'acc_x', 'acc_y',
       'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z', 'thm_1', 'thm_2', 'thm_3',
       'thm_4', 'thm_5', 'tof_1_v0', 'tof_1_v1', 'tof_1_v2', 'tof_1_v3',
       'tof_1_v4', 'tof_1_v5', 'tof_1_v6', 'tof_1_v7', 'tof_1_v8', 'tof_1_v9',
       'tof_1_v10', 'tof_1_v11', 'tof_1_v12', 'tof_1_v13', 'tof_1_v14',
       'tof_1_v15', 'tof_1_v16', 'tof_1_v17', 'tof_1_v18', 'tof_1_v19',
       'tof_1_v20', 'tof_1_v21', 'tof_1_v22', 'tof_1_v23', 'tof_1_v24',
       'tof_1_v25', 'tof_1_v26', 'tof_1_v27', 'tof_1_v28'],
      dtype='object')

# Loading Sensor Data

In [6]:
data_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'

try:
    df=pd.read_csv(data_path)
    core_required_columns=['timestamp','behavior']+[col for col in IMU_SENSOR_COLUMNS if col in df.columns]
    if not all(col in df.columns for col in core_required_columns):
        print(f"Warning: Missing some core IMU columns or 'timestamp'/'behavior'. Please check your CSV header.")
        print(f"Expected: {core_required_columns} + any thermopile/ToF columns")

    print(f"Data loaded successfully from:{data_path}")
    print("Sample Data Head:")
    print(df.head())
    print(f"Total number of samples: {len(df)}")
    print(f"Columns found in data:{df.columns.tolist()}")
    print("\n")

except FileNotFoundError:
    print(f"Error:Data file not found at '{data_path}'.")
    exit()
except Exception as e:
    print(f"An unexpected error occured while loading data:{e}")
    exit()

Expected: ['timestamp', 'behavior', 'acc_x', 'acc_y', 'acc_z'] + any thermopile/ToF columns
Data loaded successfully from:/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv
Sample Data Head:
              row_id sequence_type sequence_id  sequence_counter      subject  \
0  SEQ_000007_000000        Target  SEQ_000007                 0  SUBJ_059520   
1  SEQ_000007_000001        Target  SEQ_000007                 1  SUBJ_059520   
2  SEQ_000007_000002        Target  SEQ_000007                 2  SUBJ_059520   
3  SEQ_000007_000003        Target  SEQ_000007                 3  SUBJ_059520   
4  SEQ_000007_000004        Target  SEQ_000007                 4  SUBJ_059520   

                       orientation                                   behavior  \
0  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
1  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
2  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target 

# Feature Engineering

The purpose of this code is to convert raw time-series sensor data into a structured format suitable for training a machine learning model. This involves splitting the data into manageable parts, extracting informative statistics, and preparing labeled inputs.

1. Sliding Window with Overlap

The time-series data is divided into windows of fixed length (e.g., 50 rows), with partial overlap (e.g., 25 rows). This sliding window approach ensures that short-term patterns in the data are captured multiple times and prevents information loss that could occur at the boundaries of non-overlapping windows.

2. Filtering Valid Windows

Only windows where all rows have the same behavior label are retained. This is important for supervised learning tasks because mixed-label windows would confuse the model and reduce classification accuracy.


3. Feature Extraction

For each window and each expected sensor column, 12 statistical features are computed:

Mean

Standard deviation

Minimum and maximum

Median and variance

Root mean square (RMS)

Sum of absolute values (SMA)

Peak-to-peak (range)

Interquartile range (IQR)

Skewness

Kurtosis

4. Handling Missing Data

If a particular sensor column is missing from the data, the code fills in a vector of twelve zeroes in its place. This maintains a consistent feature vector length for each window, which is critical for downstream model training.

5. Validation and Collection

Each extracted feature vector is checked to ensure it matches the expected total dimension. Valid vectors are added to the features list, their associated labels are added to labels, and the starting index of the window is recorded in window_indices.

Once all windows are processed, the collected data is converted into NumPy arrays:

X: a 2D array of shape (num_windows, num_features) containing the feature vectors.

y: a 1D array containing the label for each window.

window_ids: a 1D array containing the starting index of each window.



1.Most machine learning algorithms require fixed-length numerical inputs. This pipeline transforms variable-length time-series data into a structured, tabular format.

2.Windowing allows you to isolate and capture short-term patterns in time-series signals, which is important for detecting behaviors or states that change over time.

3.Overlapping windows increase the amount of training data and help capture transitions more effectively. Ensuring each window has a consistent label improves model clarity and prevents noisy supervision.

In [8]:
window_size=50 #Number of data points in each window
overlap=25 #Number of overlapping data points between consecutive windows

features=[] # Extracted features for each window
labels=[] #Corresponding label for each window
window_indices=[] #Starting index of each valid window

# Helper function to extract statistical features for a single sensor series
def extract_sensor_features(series):
    if series.empty:
        # Return zeros for all features if series is empty (e.g., sensor not present)
        # This list must match the number of features extracted per sensor type below
        return [0.0] * 12 # 12 features per sensor (mean, std, min, max, median, var, rms, sma, ptp, iqr, skew, kurt)
    return [
        series.mean(), series.std(), series.min(), series.max(),
        series.median(), series.var(), np.sqrt(np.mean(series**2)),
        series.abs().sum(), np.ptp(series), iqr(series),
        skew(series), kurtosis(series)
    ]
        
    
TOTAL_FEATURE_DIMENSION = len(ALL_EXPECTED_SENSOR_COLUMNS)*12


#Sliding window loop

for i in range(0,len(df) - window_size,window_size - overlap):
    window=df.iloc[i: i + window_size]


    #Checking if window has enough data points and a consistent label
    if len(window) == window_size and window['behavior'].nunique()==1: 
        
        current_window_features=[]
        for col in ALL_EXPECTED_SENSOR_COLUMNS:
            
            if col in window.columns:
                current_window_features.extend(extract_sensor_features(window[col]))
            else:
                current_window_features.extend([0.0] * 12) #If a sensor is missing from data
                
        if len(current_window_features)==TOTAL_FEATURE_DIMENSION:
            features.append(current_window_features)
            labels.append(window['behavior'].iloc[0])
            window_indices.append(i) # Store the starting index of the window
                
        else:print(f"Warning: Feature dimension mismatch at window {i}. Expected {TOTAL_FEATURE_DIMENSION}, got {len(current_window_features)}. Skipping.")

if not features:
    print("No features extracted. Please ensure your data is large enough for the chosen window size and overlap.")
    exit()

X = np.array(features)  #shape: (num_windows,feature_size)
y = np.array(labels)    #shape:(num_windows,)
window_ids = np.array(window_indices)

print(f"Extracted {X.shape[0]} windows with {X.shape[1]} features each.")
print(f"Labels shape: {y.shape}")
print("\n")
    
                

/tmp/ipykernel_35/3009994272.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew(series), kurtosis(series)


Extracted 1484 windows with 180 features each.
Labels shape: (1484,)




# Encode Labels and Split Data

This code block is a critical stage in preparing labeled time-series data for training a machine learning model, specifically for behavior classification tasks using sensor data. It begins by validating the label array y to ensure it's a proper, non-empty NumPy array. This check is essential because a malformed or missing label array would break the entire modeling pipeline and lead to silent failures or misleading results. After confirming the validity of y, the code uses LabelEncoder to convert the original categorical behavior labels (like "walking", "running", "jumping") into integer class values (0, 1, 2, etc.). This numeric representation is required because most machine learning models cannot process string labels directly. Following this, the integer labels are passed to to_categorical, which transforms them into one-hot encoded vectors. This is crucial for classification models (especially neural networks) that use softmax outputs and categorical cross-entropy loss, as it ensures the true label is expressed as a vector with a 1 in the correct class position and 0s elsewhere.

Next, the code calculates the number of unique classes and validates whether the dataset is large and diverse enough to proceed. It explicitly checks that there are at least two distinct classes and more than one sample. This is a safety step: training on too little data or with only one class would make the model useless, or worse, allow it to overfit with no generalization power. Once the data passes these checks, the code calls train_test_split to divide the dataset into training and testing sets using a 70-30 split (test_size=0.3). It also passes in window_ids, which represent the original positions of each windowed segment in the full dataset. This can be used later to map model predictions back to time indices for visualization or analysis.

The most important design decision here is the use of stratify=y_encoded. This ensures that each class is represented proportionally in both the training and testing datasets. Without stratification, random splitting might accidentally produce a test set that has too many examples of one class and too few (or none) of another, especially if the dataset is imbalanced. Such imbalance would distort evaluation metrics like accuracy, making the model appear better than it is. Finally, the code prints the shapes of the resulting splits (X_train, X_test, y_train, y_test) to verify that the data has been split correctly and remains aligned across features, labels, and window indices. All of these steps, combined with comprehensive error handling, ensure that the model is trained and evaluated on clean, balanced, and properly structured data.



In [9]:
try:
    if not isinstance(y, np.ndarray) or y.size == 0:
        print("Error: 'y' (labels) is not a valid numpy array or is empty when trying to encode. Check data loading and feature engineering steps.")
        exit()

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    num_classes = len(label_encoder.classes_)
    print(f"Detected {num_classes} behavior classes: {label_encoder.classes_}")

    y_categorical = to_categorical(y_encoded, num_classes=num_classes)  #One-Hot Encoding

    if len(X) < 2 or num_classes < 2:
        print("Not enough samples or unique classes to split data. Need at least 2 samples and 2 unique classes.")
        print("Consider adjusting window_size/overlap or providing more data with varied behaviors.")
        exit()

    X_train, X_test, y_train, y_test, window_ids_train, window_ids_test = train_test_split(
        X, y_categorical, window_ids, test_size=0.3, random_state=42, stratify=y_encoded
    )

    print(f"Training features shape: {X_train.shape}")
    print(f"Testing features shape: {X_test.shape}")
    print(f"Training labels shape: {y_train.shape}")
    print(f"Testing labels shape: {y_test.shape}")
    print(f"Testing window IDs shape: {window_ids_test.shape}")
    print("\n")

except NameError:
    print("Critical Error: 'y' is not defined before starting label encoding. This indicates an issue in the preceding 'Feature Engineering' section or data loading, or an unstable environment.")
    print("Please ensure that your dataset exists, contains sufficient data, and has a 'behavior' column for labels.")
    exit()

except Exception as e:
    print(f"An unexpected error occurred during label encoding or data splitting: {e}")
    exit()


Detected 4 behavior classes: ['Hand at target location' 'Moves hand to target location'
 'Performs gesture' 'Relaxes and moves hand to target location']
Training features shape: (1038, 180)
Testing features shape: (446, 180)
Training labels shape: (1038, 4)
Testing labels shape: (446, 4)
Testing window IDs shape: (446,)




# Feature Scaling

Feature scaling is a crucial preprocessing step, especially when working with models like LSTMs (Long Short-Term Memory networks), which are sensitive to the magnitude and range of input values. In the given code, StandardScaler() is used to normalize the feature sets by removing the mean and scaling to unit variance. This is important because LSTMs rely on gradient-based optimization and internal gating mechanisms (like sigmoid and tanh activations) that can become unstable or ineffective when input values vary widely in scale. If one feature has values in the range of thousands while another varies only between 0 and 1, the larger feature can disproportionately influence weight updates, making training inefficient or unstable. Scaling all features to a similar range ensures that each contributes equally and the learning process converges more reliably.

The methods fit_transform() and transform() are both part of how StandardScaler (and most Scikit-learn transformers) operate. fit_transform() does two things: it first learns the parameters from the data (in this case, the mean and standard deviation of each feature) and then uses those parameters to scale the data. This is appropriate for the training set because we want the model to learn the statistics of the data it will train on. On the other hand, transform() simply applies those already-learned parameters to new data, such as the test set, ensuring that it is scaled using the same mean and standard deviation as the training data. This is critical to avoid data leakage — the situation where information from the test set influences the model during training, leading to over-optimistic performance estimates.. Correctly applying scaling not only improves model convergence and performance but also ensures the integrity of the evaluation process.



In [10]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)
print("Features scaled.")
print("\n")

Features scaled.




# Reshape Data for LSTM Input

Reshaping the feature data is necessary to prepare it for input into an LSTM model, which requires data in a 3D format: `(samples, timesteps, features)`. In the below code, the data is reshaped so that each sample contains `1` timestep and a fixed number of features (e.g., 96), resulting in shapes like `(num_samples, 1, num_features)`. This format tells the LSTM to treat each sample as a single moment in time with multiple sensor-derived statistics. Although `timesteps = 1` doesn't allow the model to learn temporal patterns across multiple time steps, it ensures compatibility with the LSTM layer. To fully leverage LSTM’s ability to capture time-based dependencies, one would use `timesteps > 1`, where each sample represents a sequence of consecutive readings over time, enabling the model to learn how features evolve across multiple points in time.


In [11]:
timesteps=1
input_features=X_train_scaled.shape[1]

X_train_reshaped=X_train_scaled.reshape(X_train_scaled.shape[0],timesteps,input_features)

X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], timesteps, input_features)

print(f"Reshaped training features shape: {X_train_reshaped.shape}")
print(f"Reshaped testing features shape: {X_test_reshaped.shape}")
print("\n")

Reshaped training features shape: (1038, 1, 180)
Reshaped testing features shape: (446, 1, 180)




# Model

| **Component**        | **Parameter / Setting**                   | **Purpose**                                         | **Reasoning**                                                                                                                                     |
| -------------------- | ----------------------------------------- | --------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------- |
| **LSTM Layer**       | `units=100`                               | Number of memory cells (neurons)                    | Controls model capacity to learn complex patterns in sequences; 100 is a common starting point.                                                   |
|                      | `input_shape=(timesteps, input_features)` | Shape of one input sample                           | Required in the first layer to define the structure: `timesteps` is the sequence length, `input_features` is the number of features per timestep. |
|                      | `activation='relu'`                       | Activation function applied to outputs              | Speeds up training and avoids vanishing gradients; although `tanh` is traditional, `relu` can work well for preprocessed inputs.                  |
| **Dropout Layer**    | `rate=0.2` (i.e., 20%)                    | Randomly disables neurons during training           | Prevents overfitting by forcing the model to not rely too heavily on any one neuron.                                                              |
| **Dense Layer**      | `units=num_classes`                       | Number of output classes                            | Produces one output per class label; used for classification.                                                                                     |
|                      | `activation='softmax'`                    | Converts logits to probabilities                    | Ensures output is interpretable as class probabilities; required for categorical classification.                                                  |
| **Compile Step**     | `optimizer='adam'`                        | Optimization algorithm                              | Adapts learning rate, handles sparse gradients, and is efficient for many types of problems.                                                      |
|                      | `loss='categorical_crossentropy'`         | Loss function for training                          | Appropriate for multi-class classification with one-hot encoded labels.                                                                           |
|                      | `metrics=['accuracy']`                    | Evaluation metric                                   | Tracks percentage of correct predictions; commonly used for classification tasks.                                                                 |
| **Training (`fit`)** | `X_train_reshaped`, `y_train`             | Input data and one-hot encoded labels               | Properly formatted data for LSTM training.                                                                                                        |
|                      | `epochs=50`                               | Number of complete training passes through the data | More epochs give the model more time to learn; 50 is typical, but needs tuning based on validation results.                                       |
|                      | `batch_size=32`                           | Number of samples per gradient update               | Smaller batches generalize better but train slower; 32 is a balanced default choice.                                                              |
|                      | `validation_split=0.1`                    | Percentage of training data used for validation     | Helps monitor performance on unseen data during training; 10% is a common choice.                                                                 |
|                      | `verbose=1`                               | Output display setting                              | Shows real-time progress bar and metrics per epoch during training.                                                                               |


In [15]:
model=Sequential()
model.add(LSTM(units=100,input_shape=(timesteps,input_features),activation='relu'))# Add an LSTM layer with 100 memory units

# Add dropout to prevent overfitting — randomly disables 20% of the LSTM's output units
model.add(Dropout(0.2))
model.add(Dense(units=num_classes,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit(
    X_train_reshaped,y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

print("\nModel training complete.")
print("\n")

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.6242 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0113 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0064 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0096 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0123 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0082 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0111 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0145 - loss: nan - val_accuracy: 0.0192 - val_loss: nan
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0

# Evaluation

The classification_report from sklearn.metrics provides a detailed breakdown of a classification model's performance across each class. It includes key evaluation metrics that help you understand how well the model is predicting each class — not just overall accuracy.

| **Metric**    | **What it Measures**                                                        | **Why It’s Useful**                                                                                  |
| ------------- | --------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------- |
| **Precision** | Out of all predicted instances of a class, how many were actually correct?  | Helps understand how many false positives the model is making. High precision = fewer false alarms.  |
| **Recall**    | Out of all actual instances of a class, how many were correctly predicted?  | Indicates how well the model captures all relevant cases (true positives). High recall = few misses. |
| **F1-score**  | Harmonic mean of precision and recall. Balances the two in a single metric. | Useful when you want a balance between precision and recall, especially in imbalanced datasets.      |
| **Support**   | Number of actual occurrences of each class in the dataset.                  | Shows how many samples of each class were present in the test set — helps interpret other metrics.   |


In [16]:
print("Evaluating the LSTM Model on the Test Set...")
loss,accuracy=model.evaluate(X_test_reshaped,y_test,verbose=0)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print("\n")

y_pred_probs=model.predict(X_test_reshaped) #shape:(num_samples,num_classes)
y_pred_classes=np.argmax(y_pred_probs,axis=1) #index of highest predicted probability
y_true_class=np.argmax(y_test,axis=1)

print("Classification Report on Test Set (LSTM):")
target_names_for_report=label_encoder.inverse_transform(np.arange(num_classes))
print(classification_report(y_true_class,y_pred_classes,target_names=target_names_for_report))
print("\n")


Evaluating the LSTM Model on the Test Set...
Test Loss: nan
Test Accuracy: 0.0112


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
Classification Report on Test Set (LSTM):
                                           precision    recall  f1-score   support

                  Hand at target location       0.02      1.00      0.03         7
            Moves hand to target location       0.00      0.00      0.00       431
                         Performs gesture       0.00      0.00      0.00         3
Relaxes and moves hand to target location       0.00      0.00      0.00         5

                                 accuracy                           0.02       446
                                macro avg       0.00      0.25      0.01       446
                             weighted avg       0.00      0.02      0.00       446





/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predictions

This code block performs real-time prediction of a behavior class from the latest window of sensor data using a trained LSTM model. It first checks if the DataFrame contains enough rows to simulate a full window (based on a defined `window_size`). If sufficient data exists, it extracts the last `window_size` rows as the input segment and computes 12 statistical features per expected sensor column, padding with zeros if any are missing. The resulting feature vector is validated for correct dimensionality, scaled using the same `StandardScaler` used during training, and reshaped to match the LSTM input format. The model then predicts class probabilities, selects the class with the highest probability, and decodes it into its original label using the label encoder. If the predicted class is not in a list of target behaviors, it is reclassified as `'non_target'`. The predicted behavior is printed; if data is insufficient or feature extraction fails, appropriate warnings are displayed instead.


In [19]:
print("Making Predictions on New Data (for a single new window)...")

if len(df) >= window_size:
    
    new_sensor_data_for_prediction=df.iloc[len(df) - window_size :] #Takes last window
    new_features_for_prediction=[]
   
    if len(new_sensor_data_for_prediction) == window_size:
        
        current_prediction_features = []
        
        for col in ALL_EXPECTED_SENSOR_COLUMNS:
            
            if col in new_sensor_data_for_prediction.columns:
                current_prediction_features.extend(extract_sensor_features(new_sensor_data_for_prediction[col]))
            else:
                current_prediction_features.extend([0.0] * 12)
                                                   
        if len(current_prediction_features) == TOTAL_FEATURE_DIMENSION:
             new_features_for_prediction.append(current_prediction_features)
            
        else:
            print(f"Warning: Prediction feature dimension mismatch. Expected {TOTAL_FEATURE_DIMENSION}, got {len(current_prediction_features)}. Skipping.")
               
    if new_features_for_prediction:
        
        X_new_prediction_scaled = scaler.transform(np.array(new_features_for_prediction))
        X_new_prediction_reshaped = X_new_prediction_scaled.reshape(1, timesteps, input_features)

        predicted_probs = model.predict(X_new_prediction_reshaped)
        predicted_class_index = np.argmax(predicted_probs, axis=1)
        predicted_behavior = label_encoder.inverse_transform(predicted_class_index)
        
        # Apply contest-specific mapping for single prediction
        if predicted_behavior[0] not in target_behaviors:
            predicted_behavior[0] = 'non_target'

        print(f"Predicted behavior for a new window of data: {predicted_behavior[0]}")
    else:
        print("Could not generate features for prediction (window size mismatch or other issue).")
else:
    print("Not enough total data samples to simulate a 'new' window for prediction.")
print("\n")

     
    

Making Predictions on New Data (for a single new window)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Predicted behavior for a new window of data: non_target




# Submission

In [20]:
predicted_behavior_names_full = label_encoder.inverse_transform(y_pred_classes)

final_submission_gestures = []
for behavior_name in predicted_behavior_names_full:
    if behavior_name in target_behaviors:
        final_submission_gestures.append(behavior_name)
    else:
        final_submission_gestures.append('non_target')

submission_df=pd.DataFrame({
    'sequence_id':window_ids_test,
    'gesture': final_submission_gestures
})

submission_df=submission_df.sort_values(by='sequence_id').reset_index(drop=True)

submission_file_name='submission_predictions_contest.csv'

submission_df.to_csv(submission_file_name,index=False)

print(f"Submission file '{submission_file_name}' created successfully." )
print("Sample of submission file head:")
print(submission_df.head())
print("\n")




Submission file 'submission_predictions_contest.csv' created successfully.
Sample of submission file head:
   sequence_id     gesture
0          325  non_target
1         1550  non_target
2         1825  non_target
3         2850  non_target
4         3625  non_target


